# Initial naive modeling
This is an effort to try creating a simple vectorfield to see if the discontinuities are plausible source locations. Each sensor location in the bounding box becomes a node with four vectors attached, one at each of the cardinal points of the compass. If the wind is blowing to the North, the vector attached to the north side of the node will be shown as pointing away from the node (an emitter) if the pollution is elevated; otherwise the vector will be shown pointing toward the node (a sink). In this scenario, we would expect to see this where P is a source of pollution and N are nodes:  
<-- N <-- P --> N -->

In [ ]:
# import libraries
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# set bounding box
UP_LEFT = # (lat, lon)
UP_RIGHT = # (lat, lon)
DOWN_RIGHT = # (lat, lon)
DOWN_LEFT = # (lat, lon)

In [ ]:
# set timeframe
START_DATE = # begin date to start taking data
END_DATE = # end date to start taking data
START_HOUR = # hour EACH DAY to start, this allows us to control for time of day effects
END_HOUR = # hour EACH DAY to end, this allows us to control for time of day effects

In [ ]:
# get data

# month_df = get_data(UP_LEFT, UP_RIGHT, DOWN_RIGHT, DOWN_LEFT, START_DATE, END_DATE, START_HOUR, END_HOUR)
# this call to get_data function that will take bounding box and timeframe and return cleaned data


In [ ]:
# calculate average counts

average_col = [] # a list that becomes a new column on our dataframe
for sensor in month_df.groupby(sensor_id):
    average = month_df.get_group(sensor_id)['2.5um/dl'].mean()
    average_col.append([average] * len(month_df.get_group(sensor_id))])
month_df['avg_2.5um/dl'] = average_col


In [ ]:
# define direction degree range
NORTH = (316,45)
EAST = (46,135)
SOUTH = (136,225)
WEST = (226,315)

In [ ]:
# go through the dataframe and add new categorical column that indicates direction: North, South, East, West, None

In [ ]:
# calculate average by compass counts

# similar to above but need to do additional groupby on new categorical column 'wind_from_compass'
# loop through each record, do a trig calculation based on angle to true direction, and sum up. 
# divide for avg and subtract the mean for the sensor from this average to get a +/- vector
# add the five different vector values in a new column called 'vector'

In [1]:
# map the sensors on google maps api

In [ ]:
# map the four vectors emerging from each direction off the sensor point